In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
import sys
import os

notebook_dir = os.getcwd()

root_dir = os.path.abspath(os.path.join(notebook_dir, "../../"))
sys.path.append(root_dir)

In [7]:
from dotenv import load_dotenv

load_dotenv()

from service import EnvironmentSecretService

In [5]:
from core.models.exchange import ExchangeType
from exchange import ExchangeFactory

In [8]:
exchange_factory = ExchangeFactory(EnvironmentSecretService())
exchange = exchange_factory.create(ExchangeType.BYBIT)
symbols = exchange.fetch_future_symbols()

In [10]:
len(symbols)

406